-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

# Решение задания 1

In [ ]:
!pip install pymorphy2

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 41.1 MB/s 
Populating the interactive namespace from numpy and matplotlib


In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
df_p = pd.read_csv(('/content/drive/MyDrive/Pyton/positive.csv'), header = None, names = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'], sep = ';', index_col=False)
df_n = pd.read_csv(('/content/drive/MyDrive/Pyton/negative.csv'), header = None, names = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'], sep = ';', index_col=False)

In [ ]:
df_p

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
114906,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
114907,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
114908,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
114909,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0


In [ ]:
df_n

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...
111918,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
111919,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
111920,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
111921,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [ ]:
df = pd.concat([df_p, df_n], axis=0)
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
111918,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
111919,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
111920,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
111921,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [ ]:
df.index = list(range(226834))
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [ ]:
# Убедимся, что нумерация правильная

df.index.unique

<bound method Index.unique of Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            226824, 226825, 226826, 226827, 226828, 226829, 226830, 226831,
            226832, 226833],
           dtype='int64', length=226834)>

In [ ]:
# Заменим признак негативных высказываний на 0

df.positive[df.positive==-1]=0
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,0,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,0,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",0,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",0,0,1,0,2516,187,265,0


In [ ]:
print('общее количество твитов', len(df))
print('количество позитивных твитов', len(df[df.positive == 1]))
print('количество негативных твитов', len(df[df.positive == -1]))

общее количество твитов 226834
количество позитивных твитов 114911
количество негативных твитов 0


----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:   
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

Для разработки моделей нам необходимо оцифровать полученные данные. Мы воспользуемся двумя методами: мешком слов [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и TF-IDF [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Но в начале необходимо разбить данные на обучающую и тестовую выборки.


# Решение задания 2

In [ ]:
# Приведем весь текст к строчным буквам

df.text = df.text.str.lower()
df.text.loc[19:22]

19    @moscow_advokat очень главное спасибо for   ме...
20    у нас есть прекрасная история, как сдохнуть за...
21    @benjamin1610 сегодня столько поводов выпить, ...
22    @stalingulag унылый?...наверное...вдруг его пе...
Name: text, dtype: object

In [ ]:
# Оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей

df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

19                    очень главное спасибо       ме...
20    у нас есть прекрасная история  как сдохнуть за...
21                  сегодня столько поводов выпить  ...
22                 унылый    наверное   вдруг его пе...
Name: text, dtype: object

In [ ]:
# Разобьем тексты на слова с помощью word_tokenize

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df.text = list(map(word_tokenize, df.text))
df.text.loc[19:22]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


19    [очень, главное, спасибо, медвед, он, работал,...
20    [у, нас, есть, прекрасная, история, как, сдохн...
21    [сегодня, столько, поводов, выпить, что, я, уж...
22    [унылый, наверное, вдруг, его, перед, этим, чп...
Name: text, dtype: object

In [ ]:
len(df.text[2])

7

In [ ]:
# Посчитаем количество слов

df_num_word = df.text.apply(len)
df_num_word.sum()

2505027

In [ ]:
# Удалим стоп-слова из наших данных

nltk.download('stopwords')
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


19    [очень, главное, спасибо, медвед, работал, кло...
20              [прекрасная, история, сдохнуть, неделю]
21    [сегодня, столько, поводов, выпить, могу, усид...
22                   [унылый, наверное, этим, чпокнули]
Name: text, dtype: object

In [ ]:
# Посчитаем количество слов

df_num_word = df.text.apply(len)
df_num_word.sum()

1493275

In [ ]:
# Проведем лемматизацию полученных слов

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медведа, работать, к...
20              [прекрасный, история, сдохнуть, неделя]
21    [сегодня, столько, повод, выпить, мочь, усидет...
22                    [унылый, наверное, это, чпокнуль]
Name: text, dtype: object

In [ ]:
# Удалим все слова, которые встречаются только 1 раз

from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text.loc[19:22]

19    очень главное спасибо медведа работать клоун а...
20                   прекрасный история сдохнуть неделя
21      сегодня столько повод выпить мочь усидеть место
22                                  унылый наверное это
Name: text, dtype: object

In [ ]:
df.to_excel('/content/drive/MyDrive/Pyton/Day05_очищенные_твиты.xlsx', index=False)

In [ ]:
df.text.loc[19:22]

19    очень главное спасибо медведа работать клоун а...
20                   прекрасный история сдохнуть неделя
21      сегодня столько повод выпить мочь усидеть место
22                                  унылый наверное это
Name: text, dtype: object

In [ ]:
# Посчитаем, сколько осталось слов

len(text_tokens), len(words_to_del), len(text_tokens) - len(words_to_del)

(1493275, 53568, 1439707)

In [ ]:
# Посчитаем пустые твиты и позитивные из них

len(df[df.text == '']), len(df[(df.text == '') & (df.positive == 1)])

(856, 382)

In [ ]:
# Удалим пустые твиты

df = df.drop(df[df.text == ''].index, axis = 0)
len(df)

225978

In [ ]:
После удаления пустых твитов у нас осталось 225978 твитов

In [ ]:
# Скопируем данные на всякий случай

df1 = df.copy()

---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

In [ ]:
# Для применения методов разобьем выборку на тестовую и тренировочную части

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=21)

---

Рассмотрим количество твитов в выборке для обучения, из них позитивных, и в выборке для теста, из них позитивных

In [ ]:
len(y_train), y_train.sum(), len(y_test), y_test.sum()

(158184, 80157, 67794, 34372)

---

### Кодировка данных   

Кодируем наши данные мешком слов и tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

---

### Классификаторы

Построим классификатор с помощью логистической регрессии:   
на основе мешка слов:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.71      0.71     33422
           1       0.72      0.73      0.72     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.80      0.79      0.79     78027
           1       0.80      0.80      0.80     80157

    accuracy                           0.80    158184
   macro avg       0.80      0.80      0.80    158184
weighted avg       0.80      0.80      0.80    158184



---

на основе tf-idf:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.73      0.69      0.71     33422
           1       0.71      0.75      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.79      0.75      0.77     78027
           1       0.77      0.80      0.79     80157

    accuracy                           0.78    158184
   macro avg       0.78      0.78      0.78    158184
weighted avg       0.78      0.78      0.78    158184



---

Видно, что модели переобучены это может быть следствием малого количества данных.

---

Построим классификатор с помощью случайного леса [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):   
на основе мешка слов:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.68      0.76      0.72     33422
           1       0.74      0.65      0.69     34372

    accuracy                           0.70     67794
   macro avg       0.71      0.71      0.70     67794
weighted avg       0.71      0.70      0.70     67794

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



---

на основе tf-idf:

In [ ]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.70      0.71     33422
           1       0.72      0.74      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



---

Получили не очень хороший результат. Модели переобучаются. Разработанные классификаторы работают недостаточно хорошо.